# List of Lists - WILDCARD Usage
Initial Model was a Map of Key:Value pairs where Value was a `Map:  {c1:{"ttl":700, "value":"abc"}, c2:{"ttl":1800, ...} , c3:{"ttl":200, .. }, ...}`

Goal: Delete all entries where `ttl < 900`

Cannot do WILDCARD search only on Map elements, but we use a sorted List of Lists

Modify data model as: `[ [ttl1, c1:{ ...}], [ttl2, c2:{ ... }], [ttl3, c3:{....} ] ]` 

The following code can then insert new entries and delete by ttl range.


### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed. 



In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;
import com.aerospike.client.cdt.ListReturnType;

import java.util.List;
import java.util.HashMap;
import java.util.Map;
import java.util.Random;
import java.util.ArrayList;
import java.util.Arrays;

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);

In [5]:
Key myKey = new Key("test", "testset", "CampaignId");
String binName = "myListBin1";
ListPolicy lPolicy = new ListPolicy(ListOrder.ORDERED, ListWriteFlags.DEFAULT);
client.delete(null,myKey);

false

In [6]:
// Initial Model was a Map of Key:Value pairs 
//Map:  {c1:{"ttl":700, "value":"abc"}, c2:{"ttl":1800, ...} , c3:{"ttl":200, .. }, ...}

// Cannot do WILDCARD search on Map values elements, so we need a Sorted List of Lists
// Modify data model as: [ [ttl1, c1:{ ...}], [ttl2, c2:{ ... }], [ttl3, c3:{....} ] ] 


//Add record data
List<Value> list_cx = new ArrayList<Value>();
list_cx.add(Value.get(1800));
HashMap <String, Value> map_cx = new HashMap <String, Value>();
map_cx.put("ttl", Value.get(1800));
map_cx.put("value", Value.get("bcd1800"));
list_cx.add(Value.get(map_cx));
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(list_cx)));
list_cx.clear();

list_cx.add(Value.get(700));
HashMap <String, Value> map_cx = new HashMap <String, Value>();
map_cx.put("ttl", Value.get(700));
map_cx.put("value", Value.get("abc700"));
list_cx.add(Value.get(map_cx));
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(list_cx)));
list_cx.clear();

list_cx.add(Value.get(200));
HashMap <String, Value> map_cx = new HashMap <String, Value>();
map_cx.put("ttl", Value.get(200));
map_cx.put("value", Value.get("cde200"));
list_cx.add(Value.get(map_cx));
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(list_cx)));
list_cx.clear();

list_cx.add(Value.get(1200));
HashMap <String, Value> map_cx = new HashMap <String, Value>();
map_cx.put("ttl", Value.get(1200));
map_cx.put("value", Value.get("efg1200"));
list_cx.add(Value.get(map_cx));
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(list_cx)));
list_cx.clear();


System.out.println("My List after data insert: "+ client.get(null, myKey));

My List after data insert: (gen:4),(exp:0),(bins:(myListBin1:[[200, {ttl=200, value=cde200}], [700, {ttl=700, value=abc700}], [1200, {ttl=1200, value=efg1200}], [1800, {ttl=1800, value=bcd1800}]]))


In [7]:
List<Value> startItem = new ArrayList<Value>();
startItem.add(Value.get(100));
startItem.add(Value.get(Value.WILDCARD));

List<Value> endItem = new ArrayList<Value>();
endItem.add(Value.get(1000));
endItem.add(Value.get(Value.WILDCARD));

System.out.println(client.operate(null, myKey, 
            ListOperation.getByValueRange("myListBin1", 
            Value.ListValue.get(startItem), Value.ListValue.get(endItem), 
            ListReturnType.VALUE)));

(gen:4),(exp:0),(bins:(myListBin1:[[200, {ttl=200, value=cde200}], [700, {ttl=700, value=abc700}]]))


In [8]:
//Check the full record
System.out.println("My record: "+ client.get(null, myKey));

My record: (gen:4),(exp:0),(bins:(myListBin1:[[200, {ttl=200, value=cde200}], [700, {ttl=700, value=abc700}], [1200, {ttl=1200, value=efg1200}], [1800, {ttl=1800, value=bcd1800}]]))


In [9]:
List<Value> startItem = new ArrayList<Value>();
startItem.add(Value.get(600));
startItem.add(Value.get(Value.WILDCARD));

List<Value> endItem = new ArrayList<Value>();
endItem.add(Value.get(1500));
endItem.add(Value.get(Value.WILDCARD));

client.operate(null, myKey, 
            ListOperation.removeByValueRange("myListBin1", 
            Value.ListValue.get(startItem), Value.ListValue.get(endItem), 
            ListReturnType.VALUE));

System.out.println("My record after ttl range delete: "+ client.get(null, myKey));

My record after ttl range delete: (gen:5),(exp:0),(bins:(myListBin1:[[200, {ttl=200, value=cde200}], [1800, {ttl=1800, value=bcd1800}]]))


# Cleanup

In [10]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"